# OWN Implementation

In [1]:
from __future__ import division

# module to read arff format files 

import arff

In [2]:
training_file="trainProdSelection.arff"

testing_file="testProdSelection.arff"

# Read File into list

In [3]:
Dtrain = []

Dtest = []

def read_file(arff_file,file_):
    
    for row in arff.load(arff_file):
        file_.append(
                
            [ 
                row['Type'],
                row['LifeStyle'],
                map(float,[row['Vacation'],row['eCredit'],row['salary'],row['property']]),
                row['label']
            ]
            
        )
        
    return file_

Dtrain=read_file(training_file,Dtrain)

Dtest=read_file(testing_file,Dtest)

    

# Normaliing the Data 

In [4]:
def normalise_data(vector):
    vacation = []
    credit = []
    salary = []
    property_ = []
    
    for y in vector:
        a,b,c,d=y[2]
        vacation.append(a)
        credit.append(b)
        salary.append(c)
        property_.append(d)
    
    normalise = lambda vec,min_,max_ : [(f-min_)/(max_-min_) for f in vec]
    
    vacation=normalise(vacation,min(vacation),max(vacation))
    
    credit = normalise(credit,min(credit),max(credit))
    
    salary = normalise(salary,min(salary),max(salary))
    
    property_ = normalise(property_,min(property_),max(property_))
    
    index = 0
    
    for l in vector:
        l[2]=[vacation[index],credit[index],salary[index],property_[index]]
        index+=1
        
    return vector

In [5]:
Dtrain = normalise_data(Dtrain)

Dtest = normalise_data(Dtest)

verification = lambda Dtest: [row_[3] for row_ in Dtest ]

validation=verification(Dtest)

# KNN implementation

In [6]:
result = []

def KNN(Dtest,Dtrain,result):
    
    import math
    
    similar =  lambda x,y: 1 if x==y else 0
    
    distance = lambda v1: math.sqrt(sum([v**2 for v in v1]))
    
    diff = lambda v0,v2,l : [v0[n]-v2[n] for n in xrange(l)]
        
    top_five = lambda vec : [vec[i][1] for i in xrange(0,5)]
    
    reverse=lambda dict_1,v:{
        v:sorted([key for key in dict_1 if dict_1[key]==v])
    }
        
    
    def eucli_distance(vec_1,vec_2):
        
        a=1-similar(vec_1[0],vec_2[0])
        
        b=1-similar(vec_1[1],vec_2[1])
        
        c,d,e,f = diff(vec_1[2],vec_2[2],len(vec_1[2]))
        
        return distance([a,b,c,d,e,f])
        
    from operator import itemgetter
    
    app_ = result.append
    
    for vec in Dtest:
        distances = []
        
        app=distances.append 
        
        for vec_ in Dtrain:
            app([eucli_distance(vec,vec_),vec_[3]])
        
        distances=sorted(distances,key=itemgetter(0))
        
        classes = top_five(distances)
        
        class_ = {
            'C1': classes.count('C1'),
            'C2': classes.count('C2'),
            'C3': classes.count('C3'),
            'C4': classes.count('C4'),
            'C5': classes.count('C5')
        }
        
        v=max(class_.values())
        
        d=reverse(class_,v)
        
        app_(d[v][0])
        
    return result
                
result=KNN(Dtest,Dtrain,result)
        

In [7]:
def accuracy(result,validation):
    
    r1 = len(result)
    
    matches = lambda v1,v2,r1: [1 if v1[r]==v2[r] else 0 for r in xrange(r1)]    
    
    return matches(result,validation,r1).count(1)/len(result)*100

accuracy(result,validation)

28.57142857142857

# Sci-Kit Learn Implementation